### Bond Pricing
(Bond yields are at Oct 10, 2019)

1-3 year: 1.55%; 3-5 year: 1.42%;  5-10 year:1.40%;  over 10 year: 1.56%

In [1]:
# Imports
import numpy as np
import pandas as pd

In [4]:
Bond = pd.read_excel(r'/Users/regina/Documents/UWO/Banking Analytics/Dataset.xlsx',
                     sheet_name = 'Bond')

def Bond_price(X,C,t): # principal, coupon, maturity
    # parameter - bond yields
    if t <= 3:
        r = 0.0155
    elif t <= 5:
        r = 0.0142
    elif t <= 10:
        r = 0.0140
    else:
        r = 0.0156
        
    # payment sequence
    payment_series = np.repeat(C*X/100, int(t)+1)
    payment_series[0] -= C*X/100
    payment_series[int(t)] += X
    
    # pv cal
    price = np.npv(r, payment_series)*(1+r)**(t-int(t))
    return(price)

# Calculation
Bond['Price'] = Bond.apply(lambda x: 
                           Bond_price(x['Principal'], x['YearlyRate'], x['Time2Maturity']),
                           axis = 1)

### Total provisions and capital requirement each operation brings

In [6]:
# Bond - provision
Bond['LGD'] = 0.45 # under fundational approach

def provision_bond(PD, LGD, EAD, N):
    P = PD*LGD*EAD*N
    return(P)

# PD floor 0.03%
Bond['EL'] = Bond.apply(lambda x:
                        provision_bond(max(0.0003, x['PD']), x['LGD'], x['Price']/100, x['BondsHeld']),
                        axis = 1)
total_provision_bond = sum(Bond['EL'])

print(total_provision_bond)

5943805883.418894


In [12]:
# Bond - capital requirement
def capital_requirement_bond(PD, LGD, M):
    from scipy.stats import norm
    b = (0.11852 - 0.05478*np.log(PD))**2
    R = 0.12*((1 - np.exp(-50*PD))/(1 - np.exp(-50)))
    R += 0.24*(1 - ((1 - np.exp(-50*PD))/(1 - np.exp(-50))))   
    
    K = norm.cdf(np.sqrt((1-R)**(-1))*norm.ppf(PD)+
                np.sqrt(R/(1-R))*norm.ppf(0.999)) - PD
    K *= LGD
    K *= (1 + (M-2.5)*b)/(1 - 1.5*b)
    return(K)
               
Bond['CapReq'] = Bond.apply(lambda x:
                            capital_requirement_bond(max(0.0003, x['PD']), x['LGD'], x['Time2Maturity']), axis=1)

In [13]:
# Bond - regulatory capital
Bond['Total'] = Bond['Price'] * Bond['BondsHeld']/100 
RWA_bond = 12.5 * np.dot(Bond['CapReq'], Bond['Total']) 
RegCap_bond = 0.115 * RWA_bond

print(RegCap_bond)

12100429777.37518


In [30]:
# RetailLoan - provision
RetailLoan = pd.read_excel(r'/Users/regina/Documents/UWO/Banking Analytics/Dataset.xlsx',
                           sheet_name = 'RetailLoan')

RetailLoan['EAD'] = RetailLoan.apply(lambda x: OutstandingAmount(x['YearlyInterestRate'],
                                                                 x['Term'],
                                                                 x['OriginalAmount'],
                                                                 x['CurrentInstallment']),axis=1)

def provision_RL(PD,EAD):
    P = PD * EAD   # LGD=100% 
    return(P)

RetailLoan['EL'] = RetailLoan.apply(lambda x: provision_RL(max(0.0003, x['PD']), x['EAD']), axis=1)
                                                                                   
total_provision_RL = sum(RetailLoan['EL'])
print(total_provision_RL)

37085132.433954544


In [31]:
# RetailLoan - capital requirement
def capital_requirement_RL(PD,LGD): # M=1
    from scipy.stats import norm
    R = 0.03 * ( (1 - np.exp(-35*PD))/(1 - np.exp(-35)))
    R += 0.16 * (1 - ((1 - np.exp(-35*PD))/(1 - np.exp(-35)))) 
    K = norm.cdf(np.sqrt((1 - R)**(-1))*norm.ppf(PD) +
                 np.sqrt(R/(1 - R))*norm.ppf(0.999)) - PD
    K *= LGD 
    return(K)

RetailLoan['LGD'] = 1
RetailLoan['CapReq'] = RetailLoan.apply(lambda x:
                                        capital_requirement_RL(max(0.0003, x['PD']), x['LGD']), axis=1)

In [32]:
# RetailLoan - regulatory capital
RWA_RL = 12.5 * np.dot(RetailLoan['CapReq'], RetailLoan['EAD']) 
RegCap_RL = 0.115 * RWA_RL

print(RegCap_RL)

26677899.2838548


In [34]:
# Total
Total_provision = total_provision_bond + total_provision_RL
Total_RegCap = RegCap_bond + RegCap_RL

# Print results
print("Total provision of bond is",total_provision_bond)
print("Total provision of retail loan is",total_provision_RL)
print("Total regulatory capital of bond is",RegCap_bond)
print("Total regulatory capital of retail loan is",RegCap_RL)
print("Total provision is",Total_provision)
print("Total regulatory capital is",Total_RegCap)

Total provision of bond is 5943805883.418894
Total provision of retail loan is 37085132.433954544
Total regulatory capital of bond is 12100429777.37518
Total regulatory capital of retail loan is 26677899.2838548
Total provision is 5980891015.852848
Total regulatory capital is 12127107676.659035
